In [1]:
#to remove warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#importing all the important libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

In [3]:
#reading necessary files
names = pd.read_table('public_dat\mimic_synthetic_feat.name',sep='\n',header=None)
data = pd.read_table('public_dat\mimic_synthetic_train.data',sep=' ',header=None)
died = pd.read_table('public_dat\mimic_synthetic_train.solution',sep='\n',header=None)
test = pd.read_table('public_dat\mimic_synthetic_test.data' ,sep=' ' ,header=None)

In [4]:
#converting the name row of excel to a list
name_col = names[0].tolist() 

In [5]:
#assigning column names to train and test data
data.drop(0,axis=1,inplace=True) 
test.drop(0,axis=1,inplace=True)
data.columns = name_col      
test.columns =name_col

In [6]:
data["MARITAL_STATUS"].fillna("MARRIED", inplace = True) 
data["ICU"].fillna("MICU", inplace = True) 
data["LANGUAGE"].fillna("ENGL", inplace = True)
test["LANGUAGE"].fillna("ENGL",inplace = True)

In [7]:
#training columns headings to a different lists acc to their datatypes
num_cols = []
cat_col = []

for cols in data.columns:
    if data[cols].dtype == object:
        cat_col.append(cols)
    else:
        num_cols.append(cols)

In [8]:
#test columns headings to a different lists acc to their datatypes
tnum_cols = []
tcat_col = []

for cols in test.columns:
    if test[cols].dtype == object:
        tcat_col.append(cols)
    else:
        tnum_cols.append(cols)

In [9]:
#filling the missing values in training data with median
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
data[num_cols] = imputer.fit_transform(data[num_cols])


In [10]:
#filling missing values in testing data with median
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
test[tnum_cols] = imputer.fit_transform(test[tnum_cols])

In [11]:
#dropping unnecessary columns
data.drop(['SUBJECT_ID', 'HADM_ID','ADMITTIME','DISCHTIME', 'LANGUAGE', 'RELIGION'], axis=1, inplace=True)
test.drop(['SUBJECT_ID', 'HADM_ID','ADMITTIME','DISCHTIME', 'LANGUAGE', 'RELIGION'], axis=1, inplace=True)

In [12]:
for name_col in data.columns:
    if data[name_col].dtypes == float:
        unique_cat = len(data[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unique categories".format(
             name_col = name_col,unique_cat = unique_cat))

Feature 'AGE' has 74' unique categories
Feature 'HR_MIN_DAY1' has 79365' unique categories
Feature 'HR_MAX_DAY1' has 79462' unique categories
Feature 'HR_MEAN_DAY1' has 79298' unique categories
Feature 'SYS_BP_MIN_DAY1' has 79329' unique categories
Feature 'SYS_BP_MAX_DAY1' has 79246' unique categories
Feature 'SYS_BP_MEAN_DAY1' has 79362' unique categories
Feature 'DIAS_BP_MIN_DAY1' has 79511' unique categories
Feature 'DIAS_BP_MAX_DAY1' has 79393' unique categories
Feature 'DIAS_BP_MEAN_DAY1' has 79387' unique categories
Feature 'MEAN_BP_MIN_DAY1' has 79409' unique categories
Feature 'MEAN_BP_MAX_DAY1' has 79512' unique categories
Feature 'MEAN_BP_MEAN_DAY1' has 79138' unique categories
Feature 'RESP_RATE_MIN_DAY1' has 79552' unique categories
Feature 'RESP_RATE_MAX_DAY1' has 79563' unique categories
Feature 'RESP_RATE_MEAN_DAY1' has 79505' unique categories
Feature 'TEMP_MIN_DAY1' has 75975' unique categories
Feature 'TEMP_MAX_DAY1' has 75272' unique categories
Feature 'TEMP_MEAN_DA

Feature 'Urinyorgca' has 1' unique categories
Feature 'Brain/nscan' has 1' unique categories
Feature 'Thyroidcncr' has 1' unique categories
Feature 'Hodgkin-sds' has 1' unique categories
Feature 'Non-Hodglym' has 1' unique categories
Feature 'Leukemias' has 1' unique categories
Feature 'Mycoses' has 2' unique categories
Feature 'Multmyeloma' has 1' unique categories
Feature 'Otprimryca' has 1' unique categories
Feature '2ndarymalig' has 2' unique categories
Feature 'Maligneopls' has 1' unique categories
Feature 'Neoplsmunsp' has 1' unique categories
Feature 'Maintchem/r' has 1' unique categories
Feature 'Bnignutneo' has 1' unique categories
Feature 'Otbnignneo' has 2' unique categories
Feature 'Thyroiddsor' has 2' unique categories
Feature 'DiabMelnoc' has 2' unique categories
Feature 'HIVinfectn' has 1' unique categories
Feature 'DiabMelw/cm' has 2' unique categories
Feature 'Otendodsor' has 2' unique categories
Feature 'Nutritdefic' has 2' unique categories
Feature 'Hyperlipidem' has

In [13]:
for name_col in test.columns:
    if data[name_col].dtypes == float:
        unique_cat = len(data[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unique categories".format(
             name_col = name_col,unique_cat = unique_cat))

Feature 'AGE' has 74' unique categories
Feature 'HR_MIN_DAY1' has 79365' unique categories
Feature 'HR_MAX_DAY1' has 79462' unique categories
Feature 'HR_MEAN_DAY1' has 79298' unique categories
Feature 'SYS_BP_MIN_DAY1' has 79329' unique categories
Feature 'SYS_BP_MAX_DAY1' has 79246' unique categories
Feature 'SYS_BP_MEAN_DAY1' has 79362' unique categories
Feature 'DIAS_BP_MIN_DAY1' has 79511' unique categories
Feature 'DIAS_BP_MAX_DAY1' has 79393' unique categories
Feature 'DIAS_BP_MEAN_DAY1' has 79387' unique categories
Feature 'MEAN_BP_MIN_DAY1' has 79409' unique categories
Feature 'MEAN_BP_MAX_DAY1' has 79512' unique categories
Feature 'MEAN_BP_MEAN_DAY1' has 79138' unique categories
Feature 'RESP_RATE_MIN_DAY1' has 79552' unique categories
Feature 'RESP_RATE_MAX_DAY1' has 79563' unique categories
Feature 'RESP_RATE_MEAN_DAY1' has 79505' unique categories
Feature 'TEMP_MIN_DAY1' has 75975' unique categories
Feature 'TEMP_MAX_DAY1' has 75272' unique categories
Feature 'TEMP_MEAN_DA

Feature 'Exam/eval' has 1' unique categories
Feature 'Otaftercare' has 2' unique categories
Feature 'Otherscreen' has 1' unique categories
Feature 'Unclassified' has 2' unique categories
Feature 'Cervixcancr' has 1' unique categories
Feature 'ECodes:Cut/pierce' has 1' unique categories
Feature 'ECodes:Drowning/submersion' has 1' unique categories
Feature 'ECodes:Fall' has 2' unique categories
Feature 'ECodes:Fire/burn' has 1' unique categories
Feature 'ECodes:Firearm' has 1' unique categories
Feature 'ECodes:Machinery' has 1' unique categories
Feature 'ecodes:motorvehicletraffic(mvt)' has 2' unique categories
Feature 'ECodes:Pedalcyclist-notMVT' has 1' unique categories
Feature 'ECodes:Pedestrian-notMVT' has 1' unique categories
Feature 'ecodes:transport-notmvt' has 1' unique categories
Feature 'ECodes:Natural/environment' has 1' unique categories
Feature 'ECodes:Overexertion' has 1' unique categories
Feature 'ECodes:Poisoning' has 1' unique categories
Feature 'ECodes:Struckby-against'

In [14]:
for name_col in data.columns:
    if data[name_col].dtypes == 'object':
        unique_cat = len(data[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unique categories".format(
             name_col = name_col,unique_cat = unique_cat))

Feature 'INSURANCE' has 5' unique categories
Feature 'MARITAL_STATUS' has 7' unique categories
Feature 'ETHNICITY' has 41' unique categories
Feature 'GENDER' has 2' unique categories
Feature 'ICU' has 5' unique categories


In [15]:
for name_col in test.columns:
    if data[name_col].dtypes == 'object':
        unique_cat = len(test[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unique categories".format(
             name_col = name_col,unique_cat = unique_cat))

Feature 'INSURANCE' has 5' unique categories
Feature 'MARITAL_STATUS' has 7' unique categories
Feature 'ETHNICITY' has 39' unique categories
Feature 'GENDER' has 2' unique categories
Feature 'ICU' has 5' unique categories


In [16]:
data["ETHNICITY"]= data["ETHNICITY"].replace("UNKNOWN/NOT_SPECIFIED", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("UNABLE_TO_OBTAIN", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("PATIENT_DECLINED_TO_ANSWER", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("AMERICAN_INDIAN/ALASKA_NATIVE_FEDERALLY_RECOGNIZED_TRIBE", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("HISPANIC/LATINO_-_HONDURAN", "OTHER")

test["ETHNICITY"]= test["ETHNICITY"].replace("UNKNOWN/NOT_SPECIFIED", "OTHER")
test["ETHNICITY"]= test["ETHNICITY"].replace("UNABLE_TO_OBTAIN", "OTHER")
test["ETHNICITY"]= test["ETHNICITY"].replace("PATIENT_DECLINED_TO_ANSWER", "OTHER")

In [17]:
imp_ft = []
nimp_ft = []
for name_col in data.columns:
    if data[name_col].dtypes == float:
        unique_cat = len(data[name_col].unique())
        if(unique_cat>1):
            imp_ft.append(name_col)
            nimp_ft.append(name_col)

In [18]:
cimp_ft = []
for name_col in data.columns:
    if data[name_col].dtypes == 'object':
        imp_ft.append(name_col)
        cimp_ft.append(name_col)

In [19]:
len(imp_ft)

162

In [20]:
fdata=data[imp_ft]

In [21]:
ftest=test[imp_ft]

In [22]:
Gender_mapping={'M':0,'F':1}                       # convert string to numeric
fdata['GENDER']=fdata['GENDER'].map(Gender_mapping)
ftest['GENDER']=ftest['GENDER'].map(Gender_mapping)

In [23]:
fdata.to_csv("mytrain.csv")   #preprocessed training file

In [24]:
ftest.to_csv("mytest.csv")    #peprocessed test file

In [25]:
died = pd.read_table('public_dat\mimic_synthetic_train.solution',sep='\n',header=None)   #solution of training data set

In [26]:
died.columns = ['DIED']

In [27]:
fdata = pd.get_dummies(fdata)   #one-hot encoding(for string variables)

In [28]:
ftest = pd.get_dummies(ftest)    #one-hot encoding

In [29]:
fdata = pd.concat([fdata,died],axis=1)    #adding died as the last column

In [30]:
from sklearn import preprocessing
#for training data
x = fdata.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
fdata = pd.DataFrame(x_scaled)              #changing the array to a Data Frame

#for test data
x = ftest.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
ftest = pd.DataFrame(x_scaled)

In [31]:
X = fdata.iloc[:,:-1]         #Train data without output class
Y = fdata.iloc[:,-1:]         #solution of train data

In [32]:
from sklearn.model_selection import train_test_split            #Split data into train & test data 
x_train_res, x_test, y_train_res, y_test = train_test_split(X, Y , test_size = 0.20, random_state = 100)

In [33]:
X=pd.DataFrame(x_train_res)
Y=pd.DataFrame(y_train_res)

In [34]:
# SMOTE
from imblearn.over_sampling import SMOTE                        # Balance imbalance data
sm = SMOTE(random_state=2)
X, Y = sm.fit_resample(X, np.ravel(Y))
print(sum(Y==0))
print(sum(Y==1))

61783
61783


In [35]:
# MODEL TESTING

In [36]:
#Logistic Regression  
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()                                    #Logistic Regression
logreg.fit(X, Y)
y_pred_logreg = logreg.predict(x_test)

acc_logreg = round(accuracy_score(y_pred_logreg, y_test) * 100, 2)      # Find Accuracy
print(acc_logreg)

78.86


In [37]:
ty_pred_logreg = logreg.predict(ftest)

In [38]:
c = 0
for i in ty_pred_logreg:
    if i == 1:
        c+=1

In [39]:
c

5396

In [40]:
over_log = pd.DataFrame(ty_pred_logreg)

In [41]:
over_log

,0
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
19996,0.0
19997,1.0
19998,0.0
19999,0.0


In [42]:
over_log.to_csv("mimic_synthetic_test.csv")   #75.76

In [43]:
#OTHER MODELS

In [44]:
# Support Vector Machines
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


svc = SVC()
svc.fit(X, Y)

SVC()

In [45]:
y_pred = svc.predict(x_test)
acc_svm = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_svm)

94.82


In [46]:
ty_pred_svm = svc.predict(ftest)

In [47]:
#Linear SVC
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(X, Y)

y_pred_linSVC = linear_svc.predict(x_test)
acc_linSVC = round(accuracy_score(y_pred_linSVC, y_test) * 100, 2)
print(acc_linSVC)

78.72


In [48]:
ty_pred_linSVC = svc.predict(ftest)

In [49]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier()
decisiontree.fit(X, Y)

y_pred_dtree = decisiontree.predict(x_test)
acc_dtree = round(accuracy_score(y_pred_dtree, y_test) * 100, 2)
print(acc_dtree)

90.29


In [50]:
ty_pred_dtree = decisiontree.predict(ftest)

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest =  RandomForestClassifier(n_estimators = 1000, criterion = 'entropy', random_state = 0)
randomforest.fit(X, Y)

y_pred_rf = randomforest.predict(x_test)
acc_rf = round(accuracy_score(y_pred_rf, y_test) * 100, 2)
print(acc_rf)

In [ ]:
ty_pred_rf = randomforest.predict(ftest)

In [ ]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(X, Y)

y_pred_gbk = gbk.predict(x_test)
acc_gbk = round(accuracy_score(y_pred_gbk, y_test) * 100, 2)
print(acc_gbk)

In [ ]:
ty_pred_gbk = gbk.predict(ftest)

In [ ]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(X, Y)

y_pred_nb = gaussian.predict(x_test)
acc_nb = round(accuracy_score(y_pred_nb, y_test) * 100, 2)
print(acc_nb)

In [ ]:
ty_pred_nb = gaussian.predict(ftest)

In [ ]:
# Perceptron
from sklearn.linear_model import Perceptron

perceptron = Perceptron()
perceptron.fit(X, Y)
y_pred_ptron = perceptron.predict(x_test)
acc_ptron = round(accuracy_score(y_pred_ptron, y_test) * 100, 2)
print(acc_ptron)

In [ ]:
ty_pred_ptron = perceptron.predict(ftest)